<a href="https://www.kaggle.com/code/yaaangzhou/playground-s3-e22-very-simple-approach?scriptVersionId=143085732" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Based on public kernals. thanks to all for your efforts.

EDA and FSS come from this [notebook](https://www.kaggle.com/code/yaaangzhou/playground-s3-e22-eda-modeling).

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

# Preprocessing
from sklearn.impute import SimpleImputer

# Models
from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings('ignore')

# Prepare data

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s3e22/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e22/test.csv')
sample_submission = pd.read_csv('/kaggle/input/playground-series-s3e22/sample_submission.csv')
origin = pd.read_csv('/kaggle/input/horse-survival-dataset/horse.csv')

# Drop column id
train.drop('id',axis=1,inplace=True)
test.drop('id',axis=1,inplace=True)

train_total = pd.concat([train, origin], ignore_index=True)
train_total.drop_duplicates(inplace=True)

total = pd.concat([train_total, test], ignore_index=True)

target = 'outcome'

In [ ]:
total.drop('lesion_3',axis=1,inplace=True)
total[target] = total[target].map({'died':0,'euthanized':1,'lived':2})
total['lesion_2'] = total['lesion_2'].apply(lambda x:1 if x>0 else 0)

In [ ]:
total = pd.get_dummies(total, columns=['surgery',
                                             'age',
                                             'temp_of_extremities',
                                             'peripheral_pulse',
                                             'mucous_membrane',
                                             'capillary_refill_time',
                                             'pain',
                                             'peristalsis',
                                             'abdominal_distention',
                                             'nasogastric_tube',
                                             'nasogastric_reflux',
                                             'rectal_exam_feces',
                                             'abdomen',
                                             'abdomo_appearance',
                                             'surgical_lesion',
                                             'cp_data'], dtype=int)

In [ ]:
df_train = total[total[target].notna()]
df_test = total[total[target].isna()]
df_test.drop(target,axis=1,inplace=True)

# Training model

In [ ]:
lgbm_md = LGBMClassifier(n_estimators=100, random_state=42).fit(df_train.drop(target,axis=1), df_train[target])
lgbm_pred = lgbm_md.predict(df_test)

# Submission

In [ ]:
sample_submission['outcome'] = lgbm_pred
sample_submission['outcome'] = sample_submission['outcome'].map({0:'died',1:'euthanized',2:'lived'})
sample_submission.to_csv('submission.csv',index=False)